<a href="https://colab.research.google.com/github/gusjav35/iageneracionimg/blob/main/generador_photomaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install diffusers transformers safetensors accelerate


In [ ]:
from PIL import Image

# Cargar imagen directamente desde el entorno de archivos
input_image = Image.open("selfie.png").convert("RGB")

In [ ]:
from diffusers import DiffusionPipeline
from google.colab import userdata
import torch

# Usar el token desde secretos
hf_token = userdata.get('HF_TOKEN')


In [ ]:
from pipeline import PhotoMakerStableDiffusionXLPipeline


In [ ]:
# Cargar modelo base compatible con PhotoMaker
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
).to("cuda" if torch.cuda.is_available() else "cpu")

# Reemplazar clase base por clase PhotoMaker
pipe.__class__ = PhotoMakerStableDiffusionXLPipeline

# Cargar el adaptador de PhotoMaker
pipe.load_photomaker_adapter(
    pretrained_model_name_or_path_or_dict="TencentARC/PhotoMaker",
    weight_name="photomaker-v1.bin",
    trigger_word="img"
)

In [ ]:
# ✅ Variable modificable: escribí aquí la especialidad médica que quieras representar
especialidad = "radiologist"  # Ejemplo: "pediatrician", "cardiologist", "surgeon", etc.

# ✅ Prompt principal: describe la escena en estilo cartoon
prompt = (
    f"a full-body cartoon illustration of a young male {especialidad} in a hospital setting, "  # Tipo de personaje y contexto
    f"actively performing tasks like examining a patient, writing on a chart, or using medical tools, "  # Acciones médicas realistas
    "wearing a medical uniform, digital art, vibrant colors, clean lines, Pixar or Disney style"  # Estilo visual tipo cartoon profesional
)

# ✅ Negative prompt: cosas que queremos evitar en la imagen generada
negative_prompt = (
    "blurry, low resolution, bad anatomy, disfigured, jpeg artifacts"  # No bloqueamos cartoon, solo evitamos fallas visuales
)

# 👉 Este código está listo para ser usado con modelos de generación de imágenes que acepten prompts, como DALL·E, Stable Diffusion, etc.



result = pipe(
    prompt=prompt,
    input_id_images=input_image,
    negative_prompt=negative_prompt,
    num_inference_steps=30,
    guidance_scale=5.0
)


# Guardar resultado
result.images[0].save("output.png")

In [ ]:
from IPython.display import Image as IPyImage, display

# Mostrar imagen generada
display(IPyImage(filename="output.png"))


In [ ]:
#Liberar memoria si ya no voy a generar imágenes
del pipe
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
